In [7]:
import pandas as pd
import sqlite3
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go


In [8]:
# Replace 'qc_data.db' with your actual database file
conn = sqlite3.connect('qc_data.db')

# Assuming a table named 'quality_measurements'
df = pd.read_sql_query("SELECT * FROM quality_measurements", conn)

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Preview the data
df.head()


,timestamp,paper_grade,machine,moisture,thickness,brightness
0,2025-01-01 00:00:00,C,M2,6.024261,130.495527,88.176814
1,2025-01-01 01:00:00,A,M2,5.584525,114.647648,90.955060
2,2025-01-01 02:00:00,C,M3,6.135228,133.173941,85.716622
3,2025-01-01 03:00:00,C,M2,5.974881,121.975996,87.612681
4,2025-01-01 04:00:00,A,M1,5.880526,140.752609,86.566355


In [9]:
grades = df['paper_grade'].unique()
machines = df['machine'].unique()
parameters = [col for col in df.columns if col not in ['timestamp', 'paper_grade', 'machine']]


In [18]:
app = Dash(__name__)
app.title = "Quality Control Dashboard"

app.layout = html.Div([
    html.H1("📊 Quality Control Dashboard" , style={'color': 'white'}),
    
    html.Div([
        html.Label("Paper Grade:" , style={'color': 'white'}),
        dcc.Dropdown(options=[{'label': g, 'value': g} for g in grades], id='grade-filter', value=grades[0]),
        
        html.Label("Machine:" , style={'color': 'white'}),
        dcc.Dropdown(options=[{'label': m, 'value': m} for m in machines], id='machine-filter', value=machines[0]),
        
        html.Label("Quality Parameter:" , style={'color': 'white'}),
        dcc.Dropdown(options=[{'label': p, 'value': p} for p in parameters], id='param-filter', value=parameters[0]),
    ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    
    dcc.Graph(id='control-chart'),
    html.Div(id='alert-box', style={'color': 'red', 'fontWeight': 'bold', 'paddingTop': '20px'})
])


In [19]:
@app.callback(
    Output('control-chart', 'figure'),
    Output('alert-box', 'children'),
    Input('grade-filter', 'value'),
    Input('machine-filter', 'value'),
    Input('param-filter', 'value')
)
def update_chart(selected_grade, selected_machine, selected_param):
    filtered_df = df[(df['paper_grade'] == selected_grade) & (df['machine'] == selected_machine)]
    param_data = filtered_df[['timestamp', selected_param]].sort_values('timestamp')

    # Calculate control limits
    mean = param_data[selected_param].mean()
    std = param_data[selected_param].std()
    ucl = mean + 3 * std
    lcl = mean - 3 * std

    # Create the control chart
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=param_data['timestamp'], y=param_data[selected_param],
                             mode='lines+markers', name='Measured'))
    fig.add_hline(y=ucl, line_dash="dash", line_color="red", annotation_text="UCL", annotation_position="top left")
    fig.add_hline(y=lcl, line_dash="dash", line_color="red", annotation_text="LCL", annotation_position="bottom left")
    fig.add_hline(y=mean, line_dash="dot", line_color="green", annotation_text="Mean", annotation_position="top right")

    fig.update_layout(title=f'{selected_param} Control Chart',
                      xaxis_title='Timestamp',
                      yaxis_title=selected_param)

    # Alerts
    out_of_spec = param_data[(param_data[selected_param] > ucl) | (param_data[selected_param] < lcl)]
    alert_msg = f"⚠️ Alert: {len(out_of_spec)} values out of control limits!" if not out_of_spec.empty else ""

    return fig, alert_msg


In [20]:
app.run(mode='inline', debug=True)


C:\Users\91855\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

